# Paso 1: Cargar el Modelo Entrenado


In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
import pickle  

# Cargar el modelo entrenado

with open('../modelos_y_vectorizadores/modelo_naive_bayes.pkl', 'rb') as archivo: 
     loaded_model = pickle.load(archivo)

# Verificar que el modelo se haya cargado correctamente
print("Modelo cargado:", loaded_model)


Modelo cargado: MultinomialNB()


In [9]:
import joblib
# Cargar el vectorizador
vectorizador = joblib.load('../modelos_y_vectorizadores/vectorizador.pkl')


# Paso 2: Preprocesar test.csv

Ahora, procede a cargar y preprocesar test.csv de la misma manera que lo hicimos con train.csv

In [3]:
import pandas as pd

# Cargar test.csv
test_df = pd.read_csv('../data_es/test.csv')

test_df.head()


,mensaje,tipo
0,Tu devolucion esta siendo procesada,ham
1,Gana dinero por compartir tus opiniones,spam
2,Recordatorio de tu cita con el nutricionista,ham
3,Quema grasa abdominal con este suplemento,spam
4,Agradecemos tu preferencia,ham


In [4]:
test_df.describe()

,mensaje,tipo
count,209,209
unique,209,2
top,Tu devolucion esta siendo procesada,ham
freq,1,109


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   mensaje  209 non-null    object
 1   tipo     209 non-null    object
dtypes: object(2)
memory usage: 3.4+ KB


In [6]:
# Contar los mensajes duplicados
duplicados = test_df[test_df.duplicated(['mensaje'], keep=False)]

# Mostrar los mensajes duplicados
print("Mensajes duplicados:")
print(duplicados)


Mensajes duplicados:
Empty DataFrame
Columns: [mensaje, tipo]
Index: []


In [7]:
# Mostrar los nombres de las columnas del dataset
print(test_df.columns)

Index(['mensaje', 'tipo'], dtype='object')


In [8]:
import nltk
from nltk.corpus import stopwords
import re

nltk.download('stopwords')
stopwords_sp = stopwords.words('spanish')

# Agregar palabras adicionales que no aporten significado en este contexto específico
stopwords_sp.extend(['este', 'nuestro', 'con', 'para', 'esta'])  # Podemos ajustar esta lista según necesitemos

# Función para limpiar el texto y filtrar stopwords
def limpiar_texto_con_stopwords(texto):
    texto = re.sub(r'\W', ' ', texto) # Eliminar caracteres no alfanuméricos
    texto = texto.lower() # Convertir a minúsculas
    texto = re.sub(r'\s+', ' ', texto) # Eliminar espacios extra
    palabras = texto.split()
    palabras_filtradas = [palabra for palabra in palabras if palabra not in stopwords_sp]
    return ' '.join(palabras_filtradas)

# Aplicar limpieza con filtro de stopwords a los mensajes
test_df['mensaje_limpio_stopwords'] = test_df['mensaje'].apply(limpiar_texto_con_stopwords)

# Mostrar los mensajes limpios con filtrado de stopwords
print(test_df[['mensaje', 'mensaje_limpio_stopwords']].head(10))

                                          mensaje  \
0             Tu devolucion esta siendo procesada   
1         Gana dinero por compartir tus opiniones   
2    Recordatorio de tu cita con el nutricionista   
3       Quema grasa abdominal con este suplemento   
4                      Agradecemos tu preferencia   
5             Obten tu score crediticio sin costo   
6    Firma el acuerdo de confidencialidad adjunto   
7         Aprovecha descuentos exclusivos para ti   
8             Confirmamos que recibimos tu pedido   
9  Desata tu potencial sexual con estas pastillas   

                 mensaje_limpio_stopwords  
0             devolucion siendo procesada  
1         gana dinero compartir opiniones  
2         recordatorio cita nutricionista  
3        quema grasa abdominal suplemento  
4                 agradecemos preferencia  
5            obten score crediticio costo  
6  firma acuerdo confidencialidad adjunto  
7         aprovecha descuentos exclusivos  
8            confirm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ferna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Vectorización de los Mensajes de Test
limpios de test_df usando el vectorizador cargado:

In [10]:
# Aplicar transformación usando el vectorizador cargado
X_test = vectorizador.transform(test_df['mensaje_limpio_stopwords'])


# Hacer Predicciones con el Modelo
El modelo Naive Bayes cargado para hacer predicciones sobre X_test:

In [19]:
from sklearn.metrics import classification_report, confusion_matrix

# Predecir con el modelo cargado
y_pred_test = loaded_model.predict(X_test)

# Convertir etiquetas numéricas a strings ('ham' y 'spam')
y_pred_str = ['spam' if pred == 1 else 'ham' for pred in y_pred_test]

# Obtener métricas de evaluación
print("Reporte de Clasificación:")
print(classification_report(test_df['tipo'], y_pred_str))

# Obtener y mostrar la matriz de confusión
print("Matriz de Confusión:")
print(confusion_matrix(test_df['tipo'], y_pred_str))


Reporte de Clasificación:
              precision    recall  f1-score   support

         ham       0.91      0.83      0.87       109
        spam       0.83      0.91      0.87       100

    accuracy                           0.87       209
   macro avg       0.87      0.87      0.87       209
weighted avg       0.87      0.87      0.87       209

Matriz de Confusión:
[[90 19]
 [ 9 91]]


# Reporte de Clasificación:

Precision: La precisión mide la proporción de predicciones positivas (spam) que fueron correctas. Para la clase 'ham' (no spam), la precisión es del 91%, lo que indica que el 91% de los mensajes clasificados como 'ham' realmente pertenecen a esa categoría. Para la clase 'spam', la precisión es del 83%, lo que significa que el 83% de los mensajes clasificados como 'spam' fueron correctos.

Recall: El recall (o sensibilidad) mide la proporción de instancias positivas que fueron correctamente identificadas por el modelo. Para la clase 'ham', el recall es del 83%, lo que indica que el modelo identificó correctamente el 83% de todos los mensajes 'ham' en el conjunto de prueba. Para la clase 'spam', el recall es del 91%, lo que significa que el modelo identificó correctamente el 91% de todos los mensajes 'spam'.

F1-score: El F1-score es la media armónica de precision y recall. Es útil cuando las clases están desbalanceadas. En tu caso, ambos F1-scores son del 87%, lo que indica un buen equilibrio entre precision y recall para ambas clases.

Support: Es el número de ocurrencias reales de cada clase en el conjunto de prueba.

# Matriz de Confusión:

La matriz de confusión muestra un resumen visual del rendimiento del modelo:

En la fila correspondiente a 'ham':

90 mensajes fueron correctamente clasificados como 'ham' (verdaderos negativos).
19 mensajes que eran 'ham' fueron incorrectamente clasificados como 'spam' (falsos positivos).
En la fila correspondiente a 'spam':

9 mensajes que eran 'spam' fueron incorrectamente clasificados como 'ham' (falsos negativos).
91 mensajes fueron correctamente clasificados como 'spam' (verdaderos positivos).

# Interpretación:

El modelo tiene una precisión general del 87% en la clasificación de mensajes ('ham' y 'spam') en el conjunto de datos de prueba.
La mayoría de los errores se concentran en la clasificación incorrecta de mensajes 'ham' como 'spam' (falsos positivos).
El recall es ligeramente más alto para 'spam' que para 'ham', lo que indica que el modelo tiende a ser más sensible a identificar mensajes 'spam'.
En resumen, el modelo muestra un rendimiento sólido en la clasificación de mensajes 'ham' y 'spam', pero podría beneficiarse de ajustes adicionales para reducir los falsos positivos.

## Comparación entre los resultads de train y test

# Precisión y Recall:

En el conjunto de entrenamiento, tanto la precisión como el recall para ambas clases ('ham' y 'spam') son ligeramente más altos en comparación con el conjunto de prueba. Esto sugiere que el modelo generaliza bien, ya que las métricas de evaluación son consistentes entre el entrenamiento y la prueba.

La precisión promedio (macro avg) es del 90% en el conjunto de entrenamiento y del 87% en el conjunto de prueba, lo cual indica una ligera caída en el rendimiento en el conjunto de prueba.

El recall promedio (macro avg) es del 90% en el conjunto de entrenamiento y del 87% en el conjunto de prueba, mostrando también una ligera disminución en el conjunto de prueba.

F1-score:

El F1-score promedio (weighted avg) es del 90% en el conjunto de entrenamiento y del 87% en el conjunto de prueba. Esta métrica combina precision y recall, por lo que también refleja una pequeña disminución en el rendimiento en el conjunto de prueba en comparación con el entrenamiento.

# Matriz de Confusión:

La matriz de confusión muestra patrones de error similares en ambos conjuntos de datos, con más falsos positivos (mensajes 'ham' clasificados incorrectamente como 'spam') que falsos negativos (mensajes 'spam' clasificados incorrectamente como 'ham') en ambos casos.

# Conclusión:

En general, el modelo muestra una buena capacidad de generalización al conjunto de datos de prueba, con métricas de evaluación sólidas y consistentes con las obtenidas durante el entrenamiento. Sin embargo, hay una ligera disminución en la precisión y recall en el conjunto de prueba, lo cual es común debido a las diferencias naturales entre los datos de entrenamiento y prueba. Esto sugiere que el modelo es robusto pero podría beneficiarse de ajustes adicionales para mejorar su desempeño en la clasificación de mensajes 'ham' y 'spam'.